In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add, coroutine_create
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

def PISO(n):
    @si.coroutine(inputs={"PI": si.Bits(n), "SI": si.Bit, "LOAD": si.Bit})
    def PISO():
        values = bits(0, n)
        O = values[-1]
        while True:
            PI, SI, LOAD = yield O
            O = values[-1]
            if LOAD:
                values = PI
            else:
                # values = [SI] + values[:-1]
                for i in range(n - 1, 0, -1):
                    values[i] = values[i - 1]
                values[0] = SI
    return PISO



@si.coroutine(inputs={"message": si.Bits(8)})
def UART_TX():
    piso = coroutine_create(PISO(8))
    message = yield
    while True:
        O = piso.send((message, 1, 1))
        O = False
        message = yield O
        for j in range(9):
            O = piso.send((message, 1, 0))
            message = yield O
        
uart = si.compile(UART_TX(), file_name="build/silica_uart.py")
with open("build/silica_uart.py", "r") as magma_file:
    print(magma_file.read())

import mantle lattice ice40
import mantle lattice mantle40
from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width):
    if "one-hot" == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
                        

In [2]:
!magma -o coreir -m coreir -t UART_TX build/silica_uart.py
!coreir -i build/silica_uart.json -o build/silica_uart.v
!yosys -p 'synth_ice40 -top UART_TX -blif build/silica_uart.blif' build/silica_uart.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_uart.txt build/silica_uart.blif
!icetime -tmd hx1k build/silica_uart.txt  | grep -B 2 "Total path delay"

In Run Generators
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!

Modified?: Yes
2.27. Printing statistics.

=== UART_TX ===

   Number of wires:                798
   Number of wire bits:           1951
   Number of public wires:         794
   Number of public wire bits:    1947
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              

In [9]:
import sys
from magma import *
from mantle import *
main = DefineCircuit("main", "message", In(Bits(8)), "O", Out(Bit))

count = CounterModM(10, 4)

shift = PISO(9)
shift(1, concat(bits(0, 1), main.message), EQ(4)(count.O, bits(0, 4)))
wire(shift.O, main.O)
EndDefine()

compile("build/magma_uart", main)

compiling FullAdder
compiling Add4Cout
compiling Register4R
compiling Counter4R
compiling Counter4Mod10
compiling Mux2
compiling Register9
compiling PISO9
compiling EQ4
compiling main
Warning (verilog): input CLK not connected to an output
Warning (verilog): input CLK not connected to an output


In [10]:
!yosys -p 'synth_ice40 -top main -blif build/magma_uart.blif' build/magma_uart.v | grep -A 15 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/magma_uart.txt build/magma_uart.blif
!icetime -tmd hx1k build/magma_uart.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== main ===

   Number of wires:                116
   Number of wire bits:            196
   Number of public wires:         116
   Number of public wire bits:     196
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 33
     SB_CARRY                        2
     SB_DFF                          9
     SB_DFFSR                        4
     SB_LUT4                        18
seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/magma_uart.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          9 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          22 / 1280
  DFF        13
  CARRY      3
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 3
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
prom